# Project imports

In [1]:
"""
All needed imports included here
"""
%load_ext autoreload
%autoreload 2
from pathlib import Path
import numpy as np
import matplotlib as plt
import torch
import pytorch_lightning as pl

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


# Data Loading step

In [2]:
"""
Create data loaders and augmentations needed here
"""
from Data.ShapeNetDataLoader import ShapeNetVoxelData
from Utils.visualization import visualize_occupancy

overfit = True

shapenet_core_path = Path("D:\ShapeNetCoreVoxel32")
shapenet_splits_csv_path = Path("Data/shapenet_splits.csv")
voxel_filename = "model_3.binvox"
# load only models from some synsets
synset_id_filter = ["04379243"]  # tables
train_data = ShapeNetVoxelData(shapenet_core_path=shapenet_core_path, shapenet_splits_csv_path=shapenet_splits_csv_path, split="train", 
    overfit=overfit, synset_id_filter=synset_id_filter, voxel_filename=voxel_filename
)
print(f"Train Set Size: {len(train_data)}")
val_data = ShapeNetVoxelData(shapenet_core_path=shapenet_core_path, shapenet_splits_csv_path=shapenet_splits_csv_path, split="val",
    overfit=overfit, synset_id_filter=synset_id_filter, voxel_filename=voxel_filename
)
print(f"Validation Set Size: {len(val_data)}")
test_data = ShapeNetVoxelData(shapenet_core_path=shapenet_core_path, shapenet_splits_csv_path=shapenet_splits_csv_path, split="test",
    overfit=overfit, synset_id_filter=synset_id_filter, voxel_filename=voxel_filename
)
print(f"Test Set Size: {len(test_data)}")

train_sample = train_data[0]
print(f'Voxel Dimensions: {train_sample.shape}')

visualize_occupancy(train_sample.squeeze(), flip_axes=True)

Train Set Size: 16
Validation Set Size: 16
Test Set Size: 16
Voxel Dimensions: (1, 32, 32, 32)


Output()

# Reconstruction Networks

In [3]:
"""
creation, training, and testing of the image2mesh reconstruction networks
"""
from Networks.Image2Mesh import Image2Voxel

model = Image2Voxel(dataloader,dataloader,dataloader,device)

X = torch.rand(10,3,32,32)
pred = model(X)

NameError: name 'dataloader' is not defined

In [ ]:
trainer = pl.Trainer(
    max_epochs=15,
    gpus=1 if torch.cuda.is_available() else None,
    profiler=True,
    progress_bar_refresh_rate=15
)

trainer.fit(model)

# Purifying predicted Meshes

In [70]:
"""
Code to purify meshes predicted by the previous networks to be used in the retrieval step
"""

'\nCode to purify meshes predicted by the previous networks to be used in the retrieval step\n'

# Mesh Encoding

In [10]:
#%env CUDA_LAUNCH_BLOCKING=1
"""
AutoEncoder Models and/or different techniques used to encode the mesh to a smaller dimensions
"""
from Networks.VoxelAutoencoder import VoxelAutoencoder
from pytorch_lightning.loggers.tensorboard import TensorBoardLogger
from pytorch_lightning.callbacks.progress import TQDMProgressBar

model = VoxelAutoencoder(train_data, val_data, test_data, device)

logger = TensorBoardLogger("tb_logs", name="my_model")
trainer = pl.Trainer(
    max_epochs=256,
    gpus=1 if torch.cuda.is_available() else None,
    log_every_n_steps=1,
    logger=logger,
    callbacks=TQDMProgressBar(refresh_rate=1)
)

trainer.fit(model)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name   | Type    | Params
-----------------------------------
0 | _model | Network | 34.2 M
-----------------------------------
34.2 M    Trainable params
0         Non-trainable params
34.2 M    Total params
136.637   Total estimated model params size (MB)


Epoch 1:  50%|█████     | 1/2 [00:00<00:00,  8.77it/s, loss=0.416, v_num=247]

ValueError: Expected parameter scale (Tensor of shape (16, 128)) of distribution Normal(loc: torch.Size([16, 128]), scale: torch.Size([16, 128])) to satisfy the constraint GreaterThan(lower_bound=0.0), but found invalid values:
tensor([[6.9622e-08, 2.4397e-10, 5.2315e+04,  ..., 3.0392e-07, 1.4562e+08,
         5.4853e-01],
        [1.0220e+08, 2.3352e-09, 5.6265e+05,  ..., 4.3202e-06, 1.4583e+09,
         1.1820e-01],
        [       inf, 4.1541e-22, 1.2662e+19,  ..., 2.3685e-01, 4.2745e-14,
         7.7411e+06],
        ...,
        [8.6447e+18, 2.2530e+06, 2.5463e+06,  ..., 5.6707e+02, 4.7845e+11,
         9.5076e+05],
        [4.2950e+05, 2.5640e+03, 1.1490e+07,  ..., 2.3736e+03, 5.9981e+08,
         9.6881e+03],
        [4.4267e+21, 2.7601e+09, 3.5591e+11,  ..., 8.1567e+01, 2.1727e+06,
         1.5470e+07]], device='cuda:0')

In [9]:
train_test_sample = train_data[0]

visualize_occupancy(train_test_sample.squeeze(), flip_axes=True)

tensor_test = torch.from_numpy(train_test_sample[np.newaxis, :])

model.eval()
with torch.no_grad():
    decoded_test = model(tensor_test)

tmp_decoded = decoded_test.clone()
tmp_decoded[decoded_test<0.5] = 0
tmp_decoded[decoded_test>=0.5] = 1

decoded_test_np = tmp_decoded.squeeze().detach().numpy()

visualize_occupancy(decoded_test_np, flip_axes=True)

Output()

Output()

# Mesh Retreival Networks

In [ ]:
"""
Models/Techniques to use the previous encoding steps to retreive objects from a specified database
"""

# Inference and Full Testing

In [ ]:
"""
Testing the entire pipeline implemented with added visualizations and discussions.
"""

# Citations

[1].....